# This is a demo that shows how tables are combined to create a schema

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import datajoint as dj

your_config_file = '../djimaging/djconfig/djconf_template.json'  # WARNING: This will not work with the empty template

assert os.path.isfile(your_config_file), 'File not found'

# Load configuration for user
dj.config.load(your_config_file)

In [ ]:
assert "database.user" in dj.config, 'Set user name'

In [ ]:
dj.config.pop('schema_name', None);  # Will use a dummy name at the end

In [ ]:
realusername = !whoami
realusername = realusername[0]

assert dj.config["database.user"] == realusername, 'Who are you?'

In [ ]:
dj.conn()

# Create schema

In [ ]:
schema = dj.schema()
schema

In [ ]:
from djimaging.core_tables import userinfo

@schema
class UserInfo(userinfo.UserInfoTemplate):
    pass

In [ ]:
from djimaging.core_tables import experiment

@schema
class Experiment(experiment.ExperimentTemplate):
    userinfo_table = UserInfo

In [ ]:
from djimaging.core_tables import field

@schema
class Field(field.FieldTemplate):
    userinfo_table = UserInfo
    experiment_table = Experiment

@schema
class Roi(field.RoiTemplate):
    field_table = Field

In [ ]:
from djimaging.core_tables import stimulus

@schema
class Stimulus(stimulus.StimulusTemplate):
    pass

In [ ]:
from djimaging.core_tables import presentation


@schema
class Presentation(presentation.PresentationTemplate):
    experiment_table = Experiment
    userinfo_table = UserInfo
    field_table = Field
    stimulus_table = Stimulus

In [ ]:
from djimaging.core_tables import location

@schema
class RelativeFieldLocation(location.RelativeFieldLocationTemplate):
    field_table = Field
    expinfo_table = Experiment.ExpInfo

@schema
class RetinalFieldLocation(location.RetinalFieldLocationTemplate):
    expinfo_table = Experiment.ExpInfo

In [ ]:
from djimaging.core_tables import traces

@schema
class Traces(traces.TracesTemplate):
    presentation_table = Presentation
    field_table = Field
    roi_table = Roi

@schema
class DetrendParams(traces.DetrendParamsTemplate):
    pass

@schema
class DetrendTraces(traces.DetrendTracesTemplate):
    presentation_table = Presentation
    detrendparams_table = DetrendParams
    traces_table = Traces

@schema
class DetrendSnippets(traces.DetrendSnippetsTemplate):
    stimulus_table = Stimulus
    presentation_table = Presentation
    traces_table = Traces
    detrendtraces_table = DetrendTraces

@schema
class Averages(traces.AveragesTemplate):
    detrendsnippets_table = DetrendSnippets

In [ ]:
schema

# Activate schema

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, schema_name=f'ageuler_{dj.config["database.user"]}_test', create_schema=True, create_tables=True)

In [ ]:
schema

In [ ]:
dj.ERD(schema)

## Now you can have a look at the tables

In [ ]:
Averages()

# Consider dropping you schema again, since this was just a test

In [ ]:
schema.drop()